# Bufstream Iceberg quickstart

In [ ]:
from pyspark.sql import SparkSession

conf = (
    pyspark.SparkConf()
        .setAppName('Jupyter')
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .set("spark.sql.catalog.bufstream-quickstart", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.bufstream-quickstart.type", "rest")
        .set("spark.sql.catalog.bufstream-quickstart.uri", "http://iceberg-rest:8181")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [ ]:
%%sql
WITH category_totals AS (
    SELECT 
        line_item.product.category.name AS category,
        SUM(line_item.quantity * line_item.unit_price_cents) / 100.0 AS total_revenue_dollars,
        COUNT(DISTINCT val.cart_id) AS carts,
        SUM(line_item.quantity) AS total_units_sold
    FROM `shopping`.`cart`
    LATERAL VIEW EXPLODE(val.line_items) AS line_item
    GROUP BY category
),
grand_total AS (
    SELECT 
        'TOTAL' AS category,
        SUM(total_revenue_dollars) AS total_revenue_dollars,
        SUM(carts) AS total_checkouts,
        SUM(total_units_sold) AS total_units_sold
    FROM category_totals
)
SELECT * FROM category_totals
UNION ALL
SELECT * FROM grand_total
ORDER BY 
    CASE WHEN category = 'TOTAL' THEN 1 ELSE 0 END,
    total_revenue_dollars DESC;
